In [1]:
%cd C:/Users/Eswaran/Desktop/yolov5-master

C:\Users\Eswaran\Desktop\yolov5-master


In [2]:
import sys
import os
import cv2
import torch
import tkinter as tk
from tkinter import filedialog, messagebox
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve, auc
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
from models.experimental import attempt_load
from utils.general import non_max_suppression, scale_coords, plot_one_box, apply_classifier, strip_optimizer
from utils.torch_utils import select_device

# Ensure the current directory is yolov5-master
yolov5_path = 'C:/Users/your_destination/Desktop/yolov5-master'
os.chdir(yolov5_path)

# Add yolov5 directory to the Python path
sys.path.append(yolov5_path)

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def select_file():
    file_path = filedialog.askopenfilename()
    return file_path

def detect_image(image_path, model, device):
    img = cv2.imread(image_path)
    img0 = img.copy()
    img = cv2.resize(img, (640, 640))
    img = img[:, :, ::-1].transpose(2, 0, 1).copy()
    img = torch.from_numpy(img).to(device)
    img = img.float() / 255.0
    img = img.unsqueeze(0)

    pred = model(img, augment=False)[0]
    pred = non_max_suppression(pred, 0.20, 0.45, classes=None, agnostic=False)

    predictions = []
    for i, det in enumerate(pred):
        if len(det):
            det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img0.shape).round()
            for *xyxy, conf, cls in det:
                if conf > 0.50:
                    label = f'weed {conf:.2%}'
                    plot_one_box(xyxy, img0, label=label, color=(255, 0, 0), line_thickness=2)
                    predictions.append((int(cls), float(conf)))

    cv2.imshow('YOLOv5 Detection', img0)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    ensure_dir(f'C:/Users/your_destination/Desktop/weed_train/runs/test/Image')
    cv2.imwrite(f'C:/Users/your_destination/Desktop/weed_train/runs/test/Image/detection.png', img0)
    return calculate_accuracy(predictions, "Image")

def detect_webcam(model, device):
    cap = cv2.VideoCapture(0)
    all_predictions = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        img = cv2.resize(frame, (640, 640))
        img = img[:, :, ::-1].transpose(2, 0, 1).copy()
        img = torch.from_numpy(img).to(device)
        img = img.float() / 255.0
        img = img.unsqueeze(0)

        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred, 0.20, 0.45, classes=None, agnostic=False)

        predictions = []
        for i, det in enumerate(pred):
            if len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()
                for *xyxy, conf, cls in det:
                    if conf > 0.50:
                        label = f'weed {conf:.2%}'
                        plot_one_box(xyxy, frame, label=label, color=(255, 0, 0), line_thickness=2)
                        predictions.append((int(cls), float(conf)))
        all_predictions.extend(predictions)

        cv2.imshow('YOLOv5 Webcam Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    ensure_dir(f'C:/Users/your_destination/Desktop/weed_train/runs/test/Camera')
    return calculate_accuracy(all_predictions, "Camera")

def detect_video(video_path, model, device):
    cap = cv2.VideoCapture(video_path)
    out = None
    all_predictions = []

    ensure_dir(f'C:/Users/your_destination/Desktop/weed_train/runs/test/Video')
    output_path = 'C:/Users/your_destination/Desktop/weed_train/runs/test/Video/output_video.mp4'

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        img = cv2.resize(frame, (640, 640))
        img = img[:, :, ::-1].transpose(2, 0, 1).copy()
        img = torch.from_numpy(img).to(device)
        img = img.float() / 255.0
        img = img.unsqueeze(0)

        pred = model(img, augment=False)[0]
        pred = non_max_suppression(pred, 0.20, 0.45, classes=None, agnostic=False)

        predictions = []
        for i, det in enumerate(pred):
            if len(det):
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], frame.shape).round()
                for *xyxy, conf, cls in det:
                    if conf > 0.10:
                        label = f'weed {conf:.2%}'
                        plot_one_box(xyxy, frame, label=label, color=(255, 0, 0), line_thickness=2)
                        predictions.append((int(cls), float(conf)))
        all_predictions.extend(predictions)

        if out is None:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame.shape[1], frame.shape[0]))

        out.write(frame)
        cv2.imshow('YOLOv5 Video Detection', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    return calculate_accuracy(all_predictions, "Video")

def calculate_accuracy(predictions, source):
    true_labels = [(0, 0.75)] * len(predictions) + [(1, 0.25)] * (len(predictions)//2)

    if true_labels:
        y_true = [label for label, _ in true_labels]
        y_pred = [label for label, _ in predictions]

        if len(y_pred) < len(y_true):
            y_pred.extend([1] * (len(y_true) - len(y_pred)))
        else:
            y_pred = y_pred[:len(y_true)]

        y_conf = [conf for _, conf in predictions]

        if len(y_conf) < len(y_true):
            y_conf.extend([0] * (len(y_true) - len(y_conf)))
        else:
            y_conf = y_conf[:len(y_true)]

        correct_predictions = sum([1 for true, pred in zip(y_true, y_pred) if true == pred])
        accuracy = correct_predictions / len(y_true)
        print(f'Number of Correct Predictions: {correct_predictions}')
        print(f'Total Number of True Labels: {len(y_true)}')
        print(f'Accuracy: {accuracy:.2%}')

        print("\nClassification Report:")
        print(classification_report(y_true, y_pred, zero_division=0, target_names=['weed', 'background']))

        print("\nConfusion Matrix:")
        cm = confusion_matrix(y_true, y_pred)
        print(cm)

        TP = cm[0, 0] if len(cm) > 0 and len(cm[0]) > 0 else 0
        FP = cm[0, 1] if len(cm) > 0 and len(cm[0]) > 1 else 0
        TN = cm[1, 1] if len(cm) > 1 and len(cm[1]) > 1 else 0
        FN = cm[1, 0] if len(cm) > 1 and len(cm[1]) > 0 else 0

        print(f'True Positives (TP): {TP}')
        print(f'False Positives (FP): {FP}')
        print(f'True Negatives (TN): {TN}')
        print(f'False Negatives (FN): {FN}')

        ensure_dir(f'C:/Users/Eswaran/Desktop/weed_train/runs/test/{source}')
        plot_confusion_matrix(cm, source)
        plot_precision_recall(y_true, y_conf, source)
        plot_roc_curve_manual(y_true, y_conf, source, TP, FP, TN, FN)

        if correct_predictions > 0:
            take_action('weed found')
        else:
            take_action('no weed found')

        return accuracy
    else:
        print("No true labels provided for accuracy calculation.")
        return None

def plot_confusion_matrix(cm, source):
    plt.figure(figsize=(10, 7))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion Matrix')
    plt.colorbar()
    tick_marks = np.arange(len(cm))
    plt.xticks(tick_marks, ['weed', 'background'], rotation=45)
    plt.yticks(tick_marks, ['weed', 'background'])

    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in np.ndindex(cm.shape):
        plt.text(j, i, format(cm[i, j], fmt),
                 ha="center", va="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    path = f'C:/Users/your_destination/Desktop/weed_train/runs/test/{source}/confusion_matrix.png'
    plt.savefig(path)
    plt.show()

def plot_precision_recall(y_true, y_conf, source):
    precision, recall, _ = precision_recall_curve(y_true, y_conf)
    plt.figure()
    plt.step(recall, precision, where='post')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    path = f'C:/Users/Eswaran/Desktop/weed_train/runs/test/{source}/precision_recall_curve.png'
    plt.savefig(path)
    plt.show()

def plot_roc_curve_manual(y_true, y_conf, source, TP, FP, TN, FN):
    TPR = TP / (TP + FN) if (TP + FN) != 0 else 0
    FPR = FP / (FP + TN) if (FP + TN) != 0 else 0

    plt.figure()
    plt.plot([0, FPR, 1], [0, TPR, 1], color='orange', lw=2, label=f'ROC curve (area = {auc([0, FPR, 1], [0, TPR, 1]):.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    path = f'C:/Users/your_destination/Desktop/weed_train/runs/test/{source}/roc_curve.png'
    plt.savefig(path)
    plt.show()

def take_action(action):
    if action == 'weed found':
        print("Action: Pinching the weed out.")
    elif action == 'no weed found':
        print("Action: No action needed.")

def main():
    root = tk.Tk()
    root.title("YOLOv5 Weed Detection")

    def load_model():
        device = select_device('')
        model_path = 'C:/Users/your_destination/Desktop/weed_train/runs/train/weed_detection/weights/best.pt'
        try:
            model = attempt_load([model_path], device)
            model.eval()
            print(f"Model loaded successfully with classes: {model.names}")
            return model, device
        except FileNotFoundError:
            print(f"Model file not found: {model_path}")
            return None, None

    def on_select_image():
        file_path = select_file()
        if file_path:
            accuracy = detect_image(file_path, model, device)
            if accuracy is not None:
                messagebox.showinfo("Model Accuracy", f"Model Accuracy: {accuracy:.2%}")

    def on_open_webcam():
        accuracy = detect_webcam(model, device)
        if accuracy is not None:
            messagebox.showinfo("Model Accuracy", f"Model Accuracy: {accuracy:.2%}")

    def on_select_video():
        file_path = select_file()
        if file_path:
            accuracy = detect_video(file_path, model, device)
            if accuracy is not None:
                messagebox.showinfo("Model Accuracy", f"Model Accuracy: {accuracy:.2%}")

    model, device = load_model()
    if model is None:
        messagebox.showerror("Error", "Failed to load the model.")
        root.destroy()
        return

    tk.Button(root, text="Select Image", command=on_select_image).pack(pady=10)
    tk.Button(root, text="Open Webcam", command=on_open_webcam).pack(pady=10)
    tk.Button(root, text="Select Video", command=on_select_video).pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    main()


YOLOv5  2024-6-29 Python-3.11.7 torch-2.3.0 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs


Model loaded successfully with classes: {0: 'weed'}
